# Getting the probability distribution of a change in the genealogical tree

We want to find the waiting distance to a change in a genealogy based on the current genealogy and on the species tree that contains it.

In [51]:
import toytree
import toyplot
import ipcoal
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
def get_tree_total_length(ttree):
    tot_len = 0
    for node_ in ttree.treenode.traverse():
        if not node_.is_root():
            tot_len += node_.dist
    return(tot_len)
def get_num_edges_at_time(tree, time):
    nodes_above = ([idx for idx, node in tree.idx_dict.items() if node.height > time])
    edges_above = len(nodes_above) + 1
    return edges_above
def get_tree_clade_times(tree):
    nodes_ = []
    heights_ = []
    for curr_node in tree.treenode.traverse():
        if not curr_node.is_leaf():
            nodes_.append(curr_node.get_leaf_names())
            heights_.append(curr_node.height)
    pddf = pd.DataFrame([nodes_,heights_],index=['clades','heights']).T
    return(pddf)
def get_branch_intervals(tr, gt, br):
    '''
    tr = species tree with Ne attribute
    gt = gene tree simulated on that species tree
    br = treenode representing a branch on the tree
    '''
    st_times = get_tree_clade_times(tr)
    gt_times = get_tree_clade_times(gt)
    coalclade = br.get_leaf_names()
    
    ###temp
    st_coal_node = tr.treenode.search_nodes(idx=tr.get_mrca_idx_from_tip_labels(br.get_leaf_names()))[0]
    nearest_st_node = st_coal_node
    while ((nearest_st_node.height + nearest_st_node.dist) < br.height):
        if nearest_st_node.is_root():
            break
        nearest_st_node = nearest_st_node.up

    coalclade = nearest_st_node.get_leaf_names()
    ###
    
    
    br_lower = br.height
    br_upper = br_lower + br.dist
    gt_clade_changes = (gt_times.heights < br_upper) & (gt_times.heights > br_lower)
    st_clade_changes = (st_times.heights < br_upper) & (st_times.heights > br_lower)
    st_time_diffed = st_times[st_clade_changes]
    #return(np.array([all(elem in clade for elem in coalclade) for clade in st_time_diffed.clades]))

    contains_clade = st_time_diffed[np.array([all(elem in clade for elem in coalclade) for clade in st_time_diffed.clades])]

    if not len(contains_clade.columns):
        contains_clade = pd.DataFrame(columns=['clades','heights'])
    contains_clade = pd.DataFrame([list(contains_clade.clades.append(pd.Series([coalclade]),ignore_index=True)),list(contains_clade.heights.append(pd.Series(br_lower)))],index=['clades','heights']).T
    contains_clade = contains_clade.sort_values('heights')

    all_members = []
    for i in contains_clade.clades:
        all_members.extend(i)
    all_members = np.unique(all_members)
    
    relevant_coals = pd.DataFrame(columns=["heights"])

    if np.sum(gt_clade_changes):
        potential_coals = gt_times[gt_clade_changes]
        relevant_coals = potential_coals[[set(i).issubset(all_members) for i in potential_coals.clades]]
        relevant_coals = relevant_coals.sort_values('heights')

    time_points = np.sort(list(contains_clade.heights) + list(relevant_coals.heights) + [br_upper])
    if int(time_points[-1]) == int(time_points[-2]):
        time_points = time_points[:-1]
    starts = time_points[:-1]
    stops = time_points[1:]
    lengths = stops-starts
    num_to_coal = np.repeat(1,len(starts))
    ne = np.repeat(1,len(starts))
    a_df = pd.DataFrame([starts,stops,lengths,num_to_coal,ne],index=['starts','stops','lengths','num_to_coal','ne']).T
    mids = (a_df.stops + a_df.starts)/2
    interval_reduced_trees=[]
    
    nes = []
    for mid in mids:
        clade = contains_clade.clades.iloc[np.sum(contains_clade.heights<mid)-1]

        cladeNe = tr.treenode.search_nodes(idx=tr.get_mrca_idx_from_tip_labels(clade))[0].Ne
        nes.append(cladeNe)
        reduced_tree = gt.prune(clade)
        interval_reduced_trees.append(reduced_tree.newick)
        
    a_df['reduced_trees'] = interval_reduced_trees
    a_df['mids'] = mids
    a_df['ne'] = nes
    a_df['num_to_coal'] = a_df.apply(lambda x: get_num_edges_at_time(toytree.tree(x['reduced_trees']), x['mids']), axis=1)
    
    return a_df

### Start with a species tree

In [2]:
# make a random tree
tre = toytree.rtree.bdtree(6,time=8e3,seed=12345)

In [3]:
# scale it so that branch lengths that make sense
tre = tre.mod.node_scale_root_height(treeheight=8e3)

### Set random Ne to each branch

In [4]:
# set a random Ne to each node
node_ne_dict = {i:np.random.randint(1,20000) for i in range(tre.nnodes)} # Ne drawn randomly between 1 and 20000
tre = tre.set_node_data('Ne',node_ne_dict)

In [5]:
tre.draw(ts='p',node_labels=True,node_sizes=15,width=500,height=500,node_mask=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7e2f405b680041b1b81bdb8ba7e9c880"> 0 1 2 3 4 5 6 7 8 9 10 r0 r1 r2 r3 r4 r5 0 2000 4000 6000 8000

### Now we can define an ipcoal model and simulate a gene tree

In [6]:
# define the model
mod = ipcoal.Model(tre,Ne=None,seed_trees=1235)
# simulate a gene tree
mod.sim_trees(1)

### Let's look at the gene tree:

In [7]:
# extract the gene tree individually
gtr = toytree.tree(mod.df.genealogy[0])
# draw it
gtr.draw(ts='p',node_labels=True,node_sizes=15,width=500,height=500,node_mask=False);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="500.0px" viewBox="0 0 500.0 500.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0ff5b7228908465da43cd7f70b67d010"> 0 1 2 3 4 5 6 7 8 9 10 r2 r1 r4 r3 r5 r0 0 4000 8000 12000 16000

### Notice that this gene tree does not match the species tree because of the short (coalescent unit) branch lengths. 

# Demonstrate how we define intervals:

### grab a node of a specific index from the gene tree

In [8]:
# grab node 4
mybranch = gtr.treenode.search_nodes(idx=4)[0]
print(mybranch)


--r5


### get the different `num_to_coal` and `ne` intervals for this branch

The coalescent probabilities will be piecewise constant, based on this table.

In [9]:
df = get_branch_intervals(tre,gtr,mybranch)
df

,starts,stops,lengths,num_to_coal,ne,reduced_trees,mids
0,0.000000,3720.299839,3720.299839,1,13685,r5:17201.6;,1860.149920
1,3720.299839,5896.649001,2176.349162,1,13685,r5:17201.6;,4808.474420
2,5896.649001,8000.000000,2103.350999,1,13685,r5:17201.6;,6948.324501
3,8000.000000,14624.489361,6624.489361,4,9600,"(r0:17201.6,(r5:15773.5,...",11312.244680
4,14624.489361,15773.536641,1149.047280,3,9600,"(r0:17201.6,(r5:15773.5,...",15199.013001


Notice there is some redundancy here. It doesn't matter, but might end up slowing down our computations a bit later on.

# Proposition 1

Given a dataframe for a branch, and a time for the recombination event, get the probability of the tree being unchanged. 

In [12]:
def calc_P_btT(t, df):
    # get the index of the interval containing t
    interval_index = np.sum(df.starts < t) - 1
    
    # get the index of the last interval on the branch
    last_index = len(df.starts)-1
    
    # sample the ai value of t-containing interval
    ai = df['num_to_coal'][interval_index]
    
    # sample the ni value of t-containing interval
    ni = df['ne'][interval_index]
    
    # sample the sig value of t-containing interval
    sigi = df['stops'][interval_index]
    
    # calculate the first term right off the bat
    first_term = (1/ai) - (1/ai)*np.exp(-1*(ai/ni)*sigi)*np.exp((ai/ni)*t)
    
    # start a variable to contain the summation for the later terms
    second_term = 0
    # loop through the intervals above the t-containing intervals
    for int_idx in range(interval_index+1,last_index+1):
        # another the summation for each of these intervals:
        internal_summation = 0
        # ask whether there is a full interval between t-containing interval and current loop interval
        if int_idx - interval_index > 1:
            # for each in-between interval......
            for q_idx in range(interval_index+1,int_idx):
                # get the a value for that interval
                aq = df['num_to_coal'][q_idx]
                # get the n value for that interval
                nq = df['ne'][q_idx]
                # get the T value for that interval
                Tq = df['lengths'][q_idx]
                # add to this internal summation this (a/n)*T
                internal_summation += ((aq/nq)*Tq)

        # define the properties of the current upper (k) interval
        # save the a value
        aint = df['num_to_coal'][int_idx]
        # save the n value
        nint = df['ne'][int_idx]
        # save the T value
        Tint = df['lengths'][int_idx]

        # calculate the expressions that are multiplied together
        # this is inside the "summation" part of the t equation that follows the first term
        first_mult = np.exp((ai/ni)*t)
        second_mult = np.exp(-1*(ai/ni)*sigi - internal_summation)
        third_mult = (1/aint)*(1-np.exp(-1*(aint/nint)*Tint))
        
        # build up the summation
        second_term += first_mult*second_mult*third_mult
    # now return the first term with the later terms (summation) added on
    return(first_term + second_term)

### Calculate it at a bunch of values of x along the branch.

In [24]:
xs = np.linspace(1, 15700,100)
ys = np.array([calc_P_btT(x,df) for x in xs])

In [28]:
toyplot.plot(xs,ys,label='prob of tree unchanging on branch b given time t',xlabel="time = t",ylabel="probability of no tree change");

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t04950e191a4c443d9117b61c9d11dd47"> <path d="M 50.031847133757964 69.4213169350661 L 55.08203049604324 73.5326567794451 L 60.132213858328505 77.6919141078406 L 65.18239722061378 81.89964739640922 L 70.23258058289906 86.15642163032228 L 75.28276394518433 90.46280837962834 L 80.3329473074696 94.81938587599959 L 85.38313066975488 99.22673909037317 L 90.43331403204014 103.6854598114958 L 95.48349739432541 108.19614672538675 L 100.53368075661069 112.75940549572374 L 105.58386411889596 117.37584884516801 L 110.63404748118123 122.04609663763657 L 115.68423084346651 126.7707759615327 L 120.73441420575179 131.55052121394723 L 125.78459756803706 136.38597418584143 L 130.83478093032232 141.2777841482212 L 135.8849642926076 146.2266079393179 L 140.9351476548929 151.23311005278285 L 145.98533101717814 156.2979627269122 L 151.03551437946342 161.42184603490895 L 156.0856977417487 166.6054479761996 L 161.13588110403398 171.84946456881357 L 166.18606446631924 177.15459994283958 L 171.23624782860452 182.5215664349711 L 176.2864311908898 187.95108468415475 L 181.33661455317505 193.44388372835158 L 186.38679791546033 199.00070110242856 L 191.4369812777456 204.62228293718815 L 196.48716464003087 210.309384059555 L 201.53734800231615 216.06276809392773 L 206.5875313646014 221.88320756471353 L 211.63771472688668 227.77148400005768 L 216.687898089172 233.72838803678127 L 221.73808145145722 239.75471952654254 L 226.78826481374253 245.85128764323534 L 231.83844817602775 252.0189109916397 L 236.88863153831306 258.2584177173379 L 241.93881490059832 264.57064561791253 L 246.9889982628836 270.9564422554399 L 252.03918162516885 277.41666507029475 L 257.08936498745413 283.9521814962815 L 262.1395483497394 290.56386907710714 L 267.1897317120247 297.2526155842118 L 272.23991507431 304.01931913597264 L 277.29009843659526 310.8648883182964 L 282.3402817988805 317.79024230661884 L 287.39046516116576 324.79631098932424 L 292.44064852345105 331.8840350926052 L 297.4908318857363 339.0543663067765 L 302.5410152480216 346.3082674140613 L 307.5911986103069 349.84254834413036 L 312.64138197259217 350.4010015231343 L 317.6915653348774 350.99759981791203 L 322.7417486971627 351.6349487286417 L 327.79193205944796 352.31583172405766 L 332.84211542173324 353.04322239758346 L 337.8922987840185 353.8202974537891 L 342.94248214630375 354.65045058188747 L 347.992665508589 355.5373072768592 L 353.04284887087437 356.4847406729338 L 358.0930322331596 357.49688845857577 L 363.1432155954448 358.57817094684816 L 368.19339895773015 359.7333103800715 L 373.24358232001543 360.9673515530856 L 378.29376568230066 362.2856838451838 L 383.343949044586 363.6940647569355 L 388.3941324068712 365.19864505469144 L 393.4443157691565 366.805995632583 L 398.4944991314418 368.5231362093284 L 403.54468249372707 370.3575659851729 L 408.5948658560123 372.31729639285084 L 413.64504921829763 374.41088608559824 L 418.69523258058285 376.64747831502217 L 423.74541594286814 379.0368408620622 L 428.7955993051534 381.5894086954342 L 433.8457826674387 384.3163295438577 L 438.8959660297239 387.22951258109094 L 443.9461493920092 390.3416804363957 L 448.99633275429454 393.666424757573 L 454.04651611657977 397.21826556923105 L 459.096699478865 401.0127146855143 L 464.14688284115033 405.0663434542378 L 469.1970662034356 409.3968551282797 L 474.2472495657209 414.02316218029875 L 479.2974329280061 418.9654688984296 L 484.3476162902914 424.2453596236727 L 489.3977996525767 429.88589301433404 L 494.447983014862 435.9117027491939

# Proposition 2

We can calculate the unchanging probability at points (t) along a branch. Now we want to integrate through all times t.

In [29]:
def calc_P_bT(df):
    last_index = len(df.starts)-1

    full_branch_summation = 0
    full_branch_start = df['starts'][0]
    full_branch_stop = df['stops'][last_index]

    for interval_index in range(len(df)):
        ai = df['num_to_coal'][interval_index]
        ni = df['ne'][interval_index]*2######################
        sigi = df['stops'][interval_index]
        sigb = df['starts'][interval_index]
        Ti = df['lengths'][interval_index]

        first_term = (1/ai)*Ti

        second_expr_second_term = 0
        for int_idx in range(interval_index+1,last_index+1): # for the *full* intervals above t
            # start with the summation
            internal_summation = 0
            if int_idx - interval_index > 1:
                for q_idx in range(interval_index+1,int_idx):
                    aq = df['num_to_coal'][q_idx]
                    nq = df['ne'][q_idx]*2############################
                    Tq = df['lengths'][q_idx]
                    internal_summation += ((aq/nq)*Tq)

            # define the properties of the current interval
            aint = df['num_to_coal'][int_idx]
            nint = df['ne'][int_idx]*2####################################
            Tint = df['lengths'][int_idx]

            # calculate the expressions that are multiplied together
            #first_mult = np.exp((ai/ni)*t)
            second_mult = np.exp(-1*(ai/ni)*sigi - internal_summation)
            third_mult = (1/aint)*(1-np.exp(-1*(aint/nint)*Tint))

            #print(second_mult*third_mult)
            second_expr_second_term += (second_mult*third_mult)*(ni/ai)

        # preventing overflow
        if ((ai/ni)*sigi < 709) and ((ai/ni)*sigb < 709): # prevent overflow...
            second_expr_second_term += -np.exp(-1*(ai/ni)*sigi) * (ni/(ai*ai))
            first_expr_second_term = (np.exp((ai/ni)*sigi) - np.exp((ai/ni)*sigb))
        # if there is no internal summation, then the problem simplifies to (e^x-e^y)/e^x , which is 1-e^(y-x)
        elif second_expr_second_term == 0:
            second_expr_second_term +=1
            first_expr_second_term = (1-np.exp((ai/ni)*sigb-(ai/ni)*sigi))* (ni/(ai*ai))
        full_branch_summation += first_term + first_expr_second_term*second_expr_second_term
    return(full_branch_summation * (1/(full_branch_stop-full_branch_start)))

### Now let's see what the probability is across the whole branch (assuming uniform probability of recombination location)

In [30]:
calc_P_bT(df)

0.21833299291045577

# Loop across branches of the genealogy to get full probability of a recombination event not changing the tree

In [31]:
def get_unchange_prob(tre, gtr):
    full_tree_length = 0
    for node in gtr.treenode.traverse():
        if not node.is_root():
            full_tree_length += node.dist
    prob_tree_unchanged = 0
    for node in gtr.treenode.traverse():
        if not node.is_root():
            #print(node.idx)
            df = get_branch_intervals(tre,gtr,node)
            #full_branch_start = df['starts'][0]
            #full_branch_stop = df['stops'].iloc[-1]

            unchanged_branch_prob = calc_P_bT(df)
            #print(unchanged_branch_prob)

            prob_tree_unchanged += (node.dist / full_tree_length) * unchanged_branch_prob
    return(prob_tree_unchanged)

In [32]:
# probability that a recombination event does not change the tree
get_unchange_prob(tre,gtr)

0.1490408666745664

## Use this to get expected wait time to a tree change.

In [35]:
def get_lambda(tre,gtr,recomb_rate):
    bigl = get_tree_total_length(gtr)
    alpha = 1-get_unchange_prob(tre, gtr)
    rho_over_2 = recomb_rate
    lambda_ = rho_over_2*alpha*bigl
    return(lambda_)

In [36]:
# lambda (rate in exponential)
get_lambda(tre, gtr,1e-9)

6.332682579213137e-05

In [37]:
# mean of exponential is the expected wait time
1/get_lambda(tre, gtr,1e-9)

15791.096229621133

### Get probability of specifc wait time:

In [39]:
lam = get_lambda(tre, gtr,1e-9)
wait_time = 10000
lam*np.exp(-lam*wait_time)

3.361729846118402e-05

# Show wait time pdf

In [44]:
toyplot.plot(np.linspace(100,17000,100),[lam*np.exp(-lam*wait_time) for wait_time in np.linspace(100,17000,100)]);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td759de1a5d5340fd9b6623fa48bdaa9f"> <path d="M 52.94117647058823 50.0 L 57.961972667855015 57.88079801692721 L 62.98276886512181 65.67686078552327 L 68.00356506238859 73.38909938894157 L 73.02436125965536 81.01841511426463 L 78.04515745692217 88.56569955783246 L 83.06595365418895 96.0318347294384 L 88.08674985145574 103.41769315540506 L 93.10754604872253 110.72413798055115 L 98.1283422459893 117.95202306906285 L 103.14913844325608 125.10219310428067 L 108.16993464052287 132.17548368741203 L 113.19073083778966 139.1727214351843 L 118.21152703505645 146.09472407644625 L 123.23232323232324 152.94230054773192 L 128.25311942959001 159.71625108779625 L 133.2739156268568 166.4173673311356 L 138.2947118241236 173.04643240050046 L 143.31550802139037 179.60422099841517 L 148.33630421865715 186.0914994977138 L 153.35710041592392 192.509026031101 L 158.37789661319073 198.8575505797512 L 163.3986928104575 205.1378150609551 L 168.4194890077243 211.35055341482285 L 173.44028520499108 217.4964916900563 L 178.46108140225786 223.5763481287978 L 183.48187759952464 229.59083325056764 L 188.50267379679144 235.540649935298 L 193.52346999405825 241.4264935054754 L 198.54426619132505 247.2490518073987 L 203.5650623885918 253.00900529156402 L 208.58585858585857 258.7070270921857 L 213.60665478312535 264.34378310586095 L 218.62745098039215 269.9199320693902 L 223.64824717765893 275.4361256367602 L 228.66904337492574 280.89300845529857 L 233.6898395721925 286.2912182410109 L 238.71063576945932 291.63138585310656 L 243.73143196672606 296.91413536772404 L 248.75222816399284 302.1400841508634 L 253.77302436125962 307.3098429305343 L 258.7938205585264 312.4240158681285 L 263.8146167557932 317.4832006290245 L 268.83541295306003 322.4879884524345 L 273.8562091503268 327.43896422049795 L 278.8770053475936 332.33670652663426 L 283.8978015448604 337.1817877431598 L 288.91859774212713 341.9747740881782 L 293.93939393939394 346.7162256917502 L 298.96019013666074 351.40669666135386 L 303.98098633392755 356.0467351466397 L 309.0017825311943 360.6368834034904 L 314.02257872846104 365.1776778573904 L 319.0433749257279 369.6696491661157 L 324.0641711229946 374.11332228174894 L 329.08496732026146 378.5092165120272 L 334.1057635175282 382.85784558103114 L 339.126559714795 387.1597176892208 L 344.14735591206176 391.41533557282656 L 349.16815210932856 395.62519656260037 L 354.18894830659536 399.78979264193725 L 359.2097445038621 403.9096105043696 L 364.230540701129 407.9851316104453 L 369.25133689839566 412.01683224399267 L 374.2721330956625 416.0051835677812 L 379.2929292929293 419.9506516785847 L 384.313725490196 423.85369766165013 L 389.3345216874629 427.71477764458297 L 394.3553178847296 431.534342850652 L 399.37611408199643 435.3128396515218 L 404.3969102792632 439.0507096194168 L 409.41770647653 442.74838957872663 L 414.4385026737968 446.4063116570543 L 419.4592988710636 450.02490333571717 L 424.48009506833034 453.6045874997041 L 429.5008912655971 457.14578248709614 L 434.5216874628639 460.6489021379548 L 439.54248366013064 464.1143558426856 L 444.5632798573975 467.5425485898815 L 449.58407605466425 470.9338810136511 L 454.60487225193106 474.288749440439 L 459.62566844919786 477.60754593534244 L 464.64646464646455 480.8906583479294 L 469.6672608437314 484.1384703575643 L 474.68805704099816 487.3513615182473 L 479.70885323826496 490.5297073029701 L 484.72964943553177 493.6738791475954 L 489.7504456327985 496.7842444942648 L 494.7712418300653 499.8611668343395 L 499.792038027

# Compare result to ipcoal simulations

In [48]:
# define the model with the same species tree
mod = ipcoal.Model(tre,Ne=None,recomb=1e-9)#seed_trees=1235
# simulate a gene tree with a really long chromosome
mod.sim_loci(nsites=100000000)

In [49]:
mod.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,5713,5713,0,0,((r1:11883.5153963039138...
1,0,5713,9417,3704,0,1,((r0:13767.0981465603490...
2,0,9417,28815,19398,0,2,((r0:13767.0981465603490...
3,0,28815,127525,98710,0,3,(r5:16974.84214048814465...
4,0,127525,136824,9299,0,4,(r5:16974.84214048814465...
...,...,...,...,...,...,...,...
9461,0,99962576,99966073,3497,0,9461,(r4:34196.24662000239186...
9462,0,99966073,99967147,1074,0,9462,(r3:21656.34706159862617...
9463,0,99967147,99972380,5233,0,9463,(r0:18136.76835850355565...
9464,0,99972380,99972570,190,0,9464,((r1:11216.8852356540064...


### Now for each simulated genealogy, let's see what our predicted length is:

In [52]:
exp_dists = []
for gen_idx in tqdm(range(len(mod.df))):
    currtree = toytree.tree(mod.df.genealogy[gen_idx])
    lam = get_lambda(tre, currtree, 1e-9)
    expected_dist = 1/lam
    # save a column of expected dists
    # this column corresponds to the observed "nbps" in mod.df
    exp_dists.append(expected_dist)

  0%|          | 0/9466 [00:00<?, ?it/s]

## Big result -- ipcoal waiting distances match our predictions:

Perform a simple check that the means match:

In [53]:
# take the ratio of observed to predicted
np.mean(mod.df.nbps/exp_dists)

1.0038351755676627

holy moly!

### Histogram of ipcoal nbps

In [54]:
# histogram of ipcoal-simulated distances
toyplot.bars(np.histogram(mod.df.nbps,bins=20));

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9af2237536994fa0aa35f6282e23b428"> 0 50000 100000 150000 0 1000 2000 3000 4000 5000

### Histogram of nbps based on expectations:

In [59]:
samples_based_on_exp_dists = np.array([np.random.exponential(i) for i in exp_dists])
# histogram of expected distances
toyplot.bars(np.histogram(samples_based_on_exp_dists,bins=20));

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600px" height="600px" viewBox="0 0 600 600" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tabe8511accef461e97f36ed165792893"> 0 50000 100000 150000 0 1000 2000 3000 4000 5000

### Next steps: topology change. This is a simple -- but tedious -- extension.